This notebook is based on [How to win over 70% matches in Rock Paper Scissors](https://towardsdatascience.com/how-to-win-over-70-matches-in-rock-paper-scissors-3e17e67e0dab)

In [ ]:
!pip install 'kaggle-environments>=0.1.6'

In [ ]:
%%writefile mysubmission.py
from __future__ import division
import random
import itertools

beat = {'R': 'P', 'P': 'S', 'S': 'R'}
_map = ['R', 'P', 'S']
_remap = {'R': 0, 'P': 1, 'S': 2}

class MarkovChain():

    def __init__(self, order, decay=1.0):
        self.decay = decay
        self.matrix = self.create_matrix(order)

    @staticmethod
    def create_matrix(order):

        def create_keys(order):
            keys = ['R', 'P', 'S']
            for i in range((order * 2 - 1)):
                key_len = len(keys)
                for i in itertools.product(keys, ''.join(keys)):
                    keys.append(''.join(i))
                keys = keys[key_len:]
            return keys

        keys = create_keys(order)
        matrix = {}
        for key in keys:
            matrix[key] = {
                'R': {'prob' : 1 / 3, 'n_obs' : 0 },
                'P': {'prob' : 1 / 3, 'n_obs' : 0 },
                'S': {'prob' : 1 / 3, 'n_obs' : 0 }
            }
        return matrix


    def update_matrix(self, pair, input):
        for i in self.matrix[pair]:
            self.matrix[pair][i]['n_obs'] = self.decay * self.matrix[pair][i]['n_obs']

        self.matrix[pair][input]['n_obs'] = self.matrix[pair][input]['n_obs'] + 1
        
        n_total = 0
        for i in self.matrix[pair]:
            n_total += self.matrix[pair][i]['n_obs']

        for i in self.matrix[pair]:
            self.matrix[pair][i]['prob'] = self.matrix[pair][i]['n_obs'] / n_total
        
        
    def predict(self, pair):
        probs = self.matrix[pair]
        probs_items = {}
        for x in probs.items():
            probs_items[x[0]] = x[1]['prob']
        max_value = max(probs_items.values())
        min_value = min(probs_items.values())
        if max_value == min_value:
            return random.choice(['R', 'P', 'S'])
        else:
            max_keys = [k for k, v in probs_items.items() if v == max_value]
            return random.choice(max_keys)

pair_diff2 = ''
pair_diff1 = ''
markov_model = MarkovChain(1, 0.9)
last_output = random.choice(['R','P','S'])

def my_agent(observation, configuration):
    global pair_diff1
    global pair_diff2
    global markov_model
    global last_output

    if observation.step > 0:
        lastOpp = _map[int(observation.lastOpponentAction)]
        pair_diff2 = pair_diff1
        pair_diff1 = last_output + lastOpp
    if pair_diff2 != '':
        markov_model.update_matrix(pair_diff2, lastOpp)
        output = beat[markov_model.predict(pair_diff1)]
    else:
        output = random.choice(['R','P','S'])
        
    last_output = output
    
    return int(_remap[output])

In [ ]:
from kaggle_environments import evaluate, make, utils
env = make("rps", debug=True)
env.render()

In [ ]:
env.reset()
env.run(["mysubmission.py", "reactionary"])
env.render(mode="ipython", width=500, height=500)

In [ ]:
env.reset()
env.run(["mysubmission.py", "copy_opponent"])
env.render(mode="ipython", width=500, height=500)

In [ ]:
env.reset()
env.run(["mysubmission.py", "counter_reactionary"])
env.render(mode="ipython", width=500, height=500)

In [ ]:
env.reset()
env.run(["mysubmission.py", "statistical"])
env.render(mode="ipython", width=500, height=500)